In [1]:
import findspark
findspark.init()

# Procesamiento de datos

import math
import pandas as pd
import numpy as np
import random 
from datetime import datetime


import pyspark.sql.functions as F
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorSizeHint, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.evaluation import ClusteringEvaluator

# Modelos de clustering 

from pyspark.ml.clustering import KMeans 

# Pyspark

sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
base_path = '.'

# Preprocesamiento

In [3]:
# Esquema

schema = StructType([
    StructField('version', StringType(), True),
    StructField('time', StringType(), True),
    StructField('id', StringType(), True),
    StructField('type', StringType(), True),
    StructField('event', StringType(), True),
    StructField('data', ArrayType(StructType([
        StructField('time',  StringType(), True),
        StructField('imei', StringType(), True),
        StructField('imsi', StringType(), True),
        StructField('rat', StringType(), True),
        ]), True))
    ])    

In [4]:
df = spark.read.schema(schema).load("../../datasets/rm_dataset/dataset_rm_v1&v2.json",
                     format="json", sep=":", inferSchema="true", header="true")
df.printSchema()

root
 |-- version: string (nullable = true)
 |-- time: string (nullable = true)
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- event: string (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- time: string (nullable = true)
 |    |    |-- imei: string (nullable = true)
 |    |    |-- imsi: string (nullable = true)
 |    |    |-- rat: string (nullable = true)



In [5]:
df.show()

+-------+----------+--------------------+----+-----+--------------------+
|version|      time|                  id|type|event|                data|
+-------+----------+--------------------+----+-----+--------------------+
|    1.0|1596800148|de539085-315f-466...|  RM| DATA|[[1596800108, 359...|
|    1.0|1596797290|de539085-315f-466...|  RM| DATA|[[1596796932, 359...|
|    1.0|1596294961|de539085-315f-466...|  RM| DATA|[[1596294959, 359...|
|    1.0|1596296581|de539085-315f-466...|  RM| DATA|[[1596296538, 352...|
|    1.0|1596297481|de539085-315f-466...|  RM| DATA|[[1596297428, 358...|
|    1.0|1596297781|de539085-315f-466...|  RM| DATA|[[1596297758, 359...|
|    1.0|1596298021|de539085-315f-466...|  RM| DATA|[[1596297972, 356...|
|    1.0|1596044533|de539085-315f-466...|  RM| DATA|[[1596039825, 355...|
|    1.0|1596295981|de539085-315f-466...|  RM| DATA|[[1596294995, 353...|
|    1.0|1596296521|de539085-315f-466...|  RM| DATA|[[1596296508, 013...|
|    1.0|1611677702|de539085-315f-466.

In [6]:
# Eliminamos columnas no útiles

df = df.drop("event","id","type","version","time")

In [7]:
# Expandimos los valores de la columna data en nuevas columnas del df.

df = df.withColumn("data", F.explode("data")).select(
  "*", F.col("data")["imei"].alias("imei"), F.col("data")["imsi"].alias("imsi"), F.col("data")["rat"].alias("rat"), 
  F.col("data")["time"].alias("time")
)
# Eliminamos la columna data tras haberla expandido.

df = df.drop("data")

In [8]:
# Cambio de formato de time

def timeFormat(time_string):
    datetime_string = datetime.utcfromtimestamp(int(time_string)).strftime('%Y-%m-%d %H:%M:%S')
    return datetime_string

timeFormatUDF = F.udf(lambda ts: timeFormat(ts)) 

df = df.withColumn("time", timeFormatUDF(F.col("time")).cast(TimestampType()))
df = df.withColumn("time", F.date_format(F.col("time"), "yyyy-MM-dd'T'HH:mm:ss.SSSZ"))

In [9]:
# Cambiar los valores NA IMEI a 000000000000000

df = df.withColumn('imei', F.when(df.imei == 'NA', '00000000000000').otherwise(df.imei))

In [10]:
df.show(50)

+--------------+---------------+---+--------------------+
|          imei|           imsi|rat|                time|
+--------------+---------------+---+--------------------+
|35937204822971|901700000015702| 2G|2020-08-07T11:35:...|
|35937204822971|901700000015700| 2G|2020-08-07T10:42:...|
|35937204822971|901700000015700| 2G|2020-08-01T15:15:...|
|35283606665993|901700000015702| 2G|2020-08-01T15:42:...|
|35818705617048|901700000015704| 2G|2020-08-01T15:57:...|
|35941008080436|901700000013634| 2G|2020-08-01T16:02:...|
|35655408691613|901700000015777| 2G|2020-08-01T16:06:...|
|35583902478151|901700000015766| 2G|2020-07-29T16:23:...|
|35583902478151|901700000015766| 2G|2020-07-29T16:26:...|
|35583902478151|901700000015766| 2G|2020-07-29T16:42:...|
|35362707574785|901700000013633| 2G|2020-08-01T15:16:...|
|35362707574785|901700000013633| 2G|2020-08-01T15:18:...|
|35362707574785|262421122334455| 2G|2020-08-01T15:26:...|
|01366500442359|901700000015708| 2G|2020-08-01T15:41:...|
|3592240792303

In [11]:
# Separamos año, mes, dia, hora, minuto y segundo de time

df = df.withColumn("year", F.year(F.col("time")))
df = df.withColumn("month", F.month(F.col("time")))
df = df.withColumn("day", F.dayofmonth(F.col("time")))
df = df.withColumn("hour", F.hour(F.col("time")))
df = df.withColumn("minute", F.minute(F.col("time")))
df = df.withColumn("second", F.second(F.col("time")))

In [12]:
# Separamos MCC, MNC y MSIN de la columna IMSI

df = df.withColumn('mcc', df.imsi.substr(1,3))
df = df.withColumn('mnc', df.imsi.substr(4,2))
df = df.withColumn('msin', df.imsi.substr(6,10))

In [13]:
""" Formato de los IMEI: TAC -- Serial_Number -- CD (15 digitos) """
# Separamos tac_a, tac_b y snr de la columna IMEI

df = df.withColumn('tac_a', df.imei.substr(1,2))
df = df.withColumn('tac_b', df.imei.substr(3,6))
df = df.withColumn('snr', df.imei.substr(9,6))

In [14]:
# Escalamos la columna YEAR con MinMaxScaler 

df = df.withColumn("year", df["year"].cast(FloatType()))
max_year = 3000
min_year = 0
df = df.withColumn("year", (F.col("year") - min_year) / (max_year - min_year))

In [15]:
# Normalizamos columnas month, day, hour, minute y second

df = df.withColumn("month", (F.col("month") - 1) / (12 - 1))
df = df.withColumn("day", (F.col("day") - 1) / (31 - 1))
df = df.withColumn("hour", (F.col("hour") - 0) / (23 - 0))
df = df.withColumn("minute", (F.col("minute") - 0) / (59 - 0))
df = df.withColumn("second", (F.col("second") - 0) / (59 - 0))

In [16]:
columns_to_transform = ['rat','mcc','mnc','msin','tac_a','tac_b','snr']

In [17]:
# Indexado de las columnas categoricas

stringIndexers = [StringIndexer(inputCol= col, outputCol=col + "_index", handleInvalid="keep") for col in columns_to_transform]

pipeline = Pipeline(stages=stringIndexers)

String_Indexer_Model = pipeline.fit(df)
df = String_Indexer_Model.transform(df)

In [18]:
# Escalado MinMax a los stringIndexer

assemblers = [VectorAssembler(inputCols=[col + '_index'], outputCol=col + "_vec") for col in columns_to_transform]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_transform]
sizeHints = [VectorSizeHint(inputCol=col + "_scaled", handleInvalid="skip", size = 1) for col in columns_to_transform]

pipeline = Pipeline(stages=assemblers + scalers + sizeHints)

minMaxScaler_Model = pipeline.fit(df)
df = minMaxScaler_Model.transform(df)

In [19]:
# Creación de vector de características

vector_assembler = VectorAssembler(inputCols=['year','month','day','hour','minute','second',
                                        'rat_scaled','mcc_scaled','mnc_scaled','msin_scaled',
                                        'tac_a_scaled','tac_b_scaled','snr_scaled'], outputCol = "features")

df = vector_assembler.transform(df)

In [20]:
df.show(1)

+--------------+---------------+---+--------------------+------------------+------------------+---+------------------+------------------+-------------------+---+---+----------+-----+------+------+---------+---------+---------+----------+-----------+-----------+---------+-------+-------+-------+--------+---------+---------+-------+----------+----------+----------+--------------------+------------+--------------------+--------------------+--------------------+
|          imei|           imsi|rat|                time|              year|             month|day|              hour|            minute|             second|mcc|mnc|      msin|tac_a| tac_b|   snr|rat_index|mcc_index|mnc_index|msin_index|tac_a_index|tac_b_index|snr_index|rat_vec|mcc_vec|mnc_vec|msin_vec|tac_a_vec|tac_b_vec|snr_vec|rat_scaled|mcc_scaled|mnc_scaled|         msin_scaled|tac_a_scaled|        tac_b_scaled|          snr_scaled|            features|
+--------------+---------------+---+--------------------+-----------------

In [21]:
df.select('features').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                     |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.6733333333333333,0.6363636363636364,0.2,0.4782608695652174,0.5932203389830508,0.13559322033898305,0.5,0.0,0.0,0.5384615384615384,0.5,0.1111111111111111,0.2222222222222222]               |
|[0.6733333333333333,0.6363636363636364,0.2,0.43478260869565216,0.711864406779661,0.2033898305084746,0.5,0.0,0.0,0.38461538461538464,0.5,0.1111111111111111,0.2222222222222222]               |
|[0.6733333333333333,0.6363636363636364,

# DistanceKmeans

In [ ]:
# Estrucutra del dataset normal

schema = StructType([
    StructField('label', StringType(), True), # no
    StructField('features', ArrayType(DoubleType()), True), # si
    ])  

df_train = spark.read.schema(schema).option("mode", "DROPMALFORMED").json(
    '../../deteccion_anomalias/preprocesamiento/KMeansStringIndexer.json')

In [ ]:
# Estrucutra del dataset normal

schema = StructType([
    StructField('rf_id', StringType(), True), # no
    StructField('time', TimestampType(), True), # si
    StructField('rat', StringType(), True), # si
    StructField('imei', StringType(), True), # si
    StructField('imsi', StringType(), True), # si
    StructField('label', StringType(), True), # no
    StructField('year', DoubleType(), True), # si
    StructField('month', DoubleType(), True), # si
    StructField('day', DoubleType(), True), # si
    StructField('hour', DoubleType(), True), # si
    StructField('minute', DoubleType(), True), # si
    StructField('second', DoubleType(), True), # si
    StructField('mcc', StringType(), True), # si
    StructField('mnc', StringType(), True), # si
    StructField('msin', StringType(), True), # si
    StructField('tac_a', StringType(), True), # no
    StructField('tac_b', StringType(), True), # no
    StructField('snr', StringType(), True), # no
    StructField('cd', StringType(), True), # si
    StructField('rat_ohe', ArrayType(DoubleType()), True), # si
    StructField('cat_imsi', ArrayType(StringType()), True), # si
    StructField('cat_imei', ArrayType(StringType()), True), # si
    StructField('imei_vec', ArrayType(DoubleType()), True), # si
    StructField('imsi_vec', ArrayType(DoubleType()), True), # si
    StructField('features', ArrayType(DoubleType())), # si
    ])  

df_train = spark.read.schema(schema).option("mode", "DROPMALFORMED").json(
    '../../deteccion_anomalias/preprocesamiento/KMeansWord2Vec.json')

In [ ]:
# Función de calculo de pertenencia a un cluster

def centroid (k,centers):
    return centers[k].tolist()

In [ ]:
# Función de calculo de distancia euclidea al centroid

def distToCentroid(datapt, centroid):
    return math.sqrt(Vectors.squared_distance(datapt, centroid))

In [ ]:
new_schema = ArrayType(DoubleType(), containsNull=False)
udf_foo = F.udf(lambda x:x, new_schema)

In [ ]:
# Separamos df en datos normales

df_normal = df_train.filter(df_train.label == 'Normal')
df_normal = df_normal.withColumn("features",udf_foo("features"))

In [ ]:
df_normal.show(1, truncate = False)

In [ ]:
# Entrenamiento y predicción

kmeans = KMeans(k=30, maxIter=100, tol=1e-4, seed=319869)
model = kmeans.fit(df_normal.select('features'))

In [ ]:
model_output_path = "{}/data/distanceKmeansRmSIModel.bin".format(base_path)
# model_output_path = "{}/data/distanceKmeansRmW2VModel.bin".format(base_path)
model.write().overwrite().save(model_output_path)

In [ ]:
df_normal = model.transform(df_normal)

In [ ]:
centers = model.clusterCenters()

In [ ]:
vectorCent = F.udf(lambda k: centroid(k,centers), ArrayType(DoubleType()))
euclDistance = F.udf(lambda data,centroid: distToCentroid(data,centroid),FloatType())

In [ ]:
# Calculamos valor del centroid más cercano.

df_normal = df_normal.withColumn('centroid', vectorCent(F.col('prediction')))

In [ ]:
# Calculamos distancia al centroid más cercano.

df_normal = df_normal.withColumn('distance', euclDistance(F.col('features'),F.col('centroid')))

In [ ]:
threshold = df_normal.groupBy('prediction').agg(F.sort_array(F.collect_list('distance'), asc=False).alias('distances'))\
.orderBy('prediction')

In [ ]:
threshold = threshold.select('distances').toPandas()['distances'].values

In [ ]:
threshold_path = '{}/data/thresholdStringIndexer.npy'.format(base_path)
# threshold_path = '{}/data/thresholdWord2Vec.npy'.format(base_path)
np.save(threshold_path, threshold)